In [1]:
# Import the needed Packages
# FMU Simulation
import MoBASimulator as mb
# Numpy
import numpy as np
# Bokeh for Plotting
import bokeh.plotting as bk
import bokeh.io as bi
#from bokeh.io import export_svgs
#bi.output_notebook()
# Algorithms
import Algorithms as alg


# Model Parameter
k = [[10.,.5],[5.,8.]]
l = [[1e-10,1e-10],[1e-10,1e-10]]
# Generate stable polynomial with roots between -1/1....-1/20 rad/s 
r = np.random.uniform(-1.0,-8.,(2,2,8))
t = np.zeros((2,2,9))
for outputs in range(0,2):
    for inputs in range(0,2):
        t[outputs,inputs,:] = np.polynomial.polynomial.polyfromroots(r[outputs,inputs,:])

# The needed Parameter
K = np.zeros((2,2))
T = np.zeros((2,2))
L = np.zeros((2,2))

# Load a Model
sim = mb.Simulator()
sim.clear()
sim.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2_n9/Masterthesis_Models_mimo_0processmodel.fmu")
sim.setOperationMode('FMU for ModelExchange')

# Show log window
sim.showLogWindow()

# Parameter Values
params = {}
# Loop over system size
for outputs in range(1,3):
    for inputs in range(1,3):
        # Process Parameter 
        # System Gain
        params.update({"fmu.num["+str(outputs)+","+str(inputs)+",1]": k[outputs-1][inputs-1]})
        # System Lag
        for order in range(0,9):
            params.update({"fmu.den["+str(outputs)+","+str(inputs)+","+str(order+1)+"]": t[outputs-1][inputs-1][order]})
        #params.update({"fmu.den["+str(outputs)+","+str(inputs)+",9]": 1})
        # System Delay
        params.update({"fmu.delay["+str(outputs)+","+str(inputs)+"]": l[outputs-1][inputs-1]})

# Set Parameter and show for checking
sim.set(params)

# Get the state space representation
ss = sim.analyser_getStateSpaceForm()

# Identify

# First run, Input 1 -> Output 1 & 2
sim.set({"fmu.u[1]": 1,"fmu.u[2]": 0})
# Set timestep = 1e-2, endtime = 100
res=sim.simulate(0.01, 500)

# Get the signals
y = res["fmu.y[1]"]
y2 = res["fmu.y[2]"]
u = res["fmu.u[1]"]
time = res["time"]

# Get TF from Input 1 to Output 1
K[0][0],T[0][0],L[0][0]=alg.Integral_Identification(y,u,time)
# Get TF from Input 1 to Output 2
K[1][0],T[1][0],L[1][0]=alg.Integral_Identification(y2,u,time)


# Second run, Input 2 -> Output 1 & 2
# Reload the Model to set everything to zero
sim.resetModelState()
sim.set({"fmu.u[1]":0, "fmu.u[2]":1})
# Set timestep = 1e-2, endtime = 100
res=sim.simulate(0.01, 500)

# Get the signals
y = res["fmu.y[1]"]
y2 = res["fmu.y[2]"]
u = res["fmu.u[2]"]
time = res["time"]

# Get TF from Input 2 to Output 1
K[0][1],T[0][1],L[0][1] = alg.Integral_Identification(y,u,time)
# Get TF from Input 2 to Output 2
K[1][1],T[1][1],L[1][1] = alg.Integral_Identification(y2,u,time)
K,T,L

(array([[ 10.00003948,   0.50000775],
        [  5.34804919,   8.00043588]]), array([[ 16.21623268,  16.1089342 ],
        [ 12.60384402,  18.71524775]]), array([[ 23.2465355 ,  23.88840385],
        [ 17.37778634,  28.76586772]]))

In [2]:
# Make a controller
#KY,B,D = alg.Control_Decentral(K,T,L)
#KY,B,D = alg.Control_Astrom(K,T,L,0.01*np.eye(2,2))
KY,B,D = alg.Control_Decoupled(K,T,L,0.01*np.eye(2,2))

In [3]:
# Make KPR,KPY, KIR and KIY
KPR = np.zeros((2,2))
KPY = np.zeros((2,2))
KIR = np.zeros((2,2))
KIY = np.zeros((2,2))
# Fill with values
for outputs in range(0,2):
    for inputs in range(0,2):
        # Proportional Controller
        KPY[outputs,inputs] = KY[outputs,inputs,0]
        # Intergral Controller
        KIY[outputs,inputs] = KY[outputs,inputs,1]

# Implement Set-point Weight
KPR = np.dot(B,KPY)
KIR = KIY

In [4]:
# Store the Matrices
A = ss['A']
B = ss['B']
C = ss['C']
DSys = ss['D'] # Name different since decoupler D
I = np.eye(A.shape[0])
# Array for singular values
sv = np.zeros((2,1000))
counter = 0

# Frequency Spectrum
omega = np.logspace(-10,3,10000)
# Singular Value Storage for sensitivity
sv_s = np.zeros((2,omega.shape[0]))
# Singular Value Storage for complementary sensitivity
sv_t = np.zeros((2,omega.shape[0]))

# Counter for storage
counter = 0
for freq in omega:
    G = np.dot(np.dot(C,np.linalg.inv(freq*1j*I-A)),B)+DSys
    S = np.linalg.inv(np.eye(2,2)-np.dot(G,np.add(KPY,1/(freq*1j)*KIY)))
    u,sv_s[:,counter],w = np.linalg.svd(np.abs(S))
    T = np.dot(S,np.dot(G, np.add(KPR,1/(freq*1j)*KIR) ))
    u,sv_t[:,counter],w = np.linalg.svd(np.abs(T))
    counter = counter +1


# Make Transfer function Matrix for frequency
p = bk.figure(title = "Singular values - Sensitivity Transfer Function", y_axis_type = "log",  x_axis_type = "log")
p.line(omega,sv_s[0,:], color = "blue")
p.line(omega,sv_s[1,:], color = "red")
bk.show(p)

p = bk.figure(title = "Singular values - Complementary Sensitivity Transfer Function", y_axis_type = "log",  x_axis_type = "log")
p.line(omega,sv_t[0,:], color = "blue")
p.line(omega,sv_t[1,:], color = "red")
bk.show(p)

In [5]:
# Model Parameter
k = [[10.,.5],[5.,8.]]
l = [[1e-10,1e-10],[1e-10,1e-10]]
# Generate stable polynomial with roots between -1/1....-1/20 rad/s 
r = np.random.uniform(-1.0,-8.,(2,2,8))
t = np.zeros((2,2,9))
for outputs in range(0,2):
    for inputs in range(0,2):
        t[outputs,inputs,:] = np.polynomial.polynomial.polyfromroots(r[outputs,inputs,:])

# The needed Parameter
K = np.zeros((2,2))
T = np.zeros((2,2))
L = np.zeros((2,2))

# Load a Model
sim = mb.Simulator()
sim.clear()
sim.loadModel("C:/Users/juliu/Documents/Thesis/Modelica/FMU/2_2_n9/Masterthesis_Models_mimo_0processmodel.fmu")
sim.setOperationMode('FMU for ModelExchange')

# Show log window
sim.showLogWindow()

# Parameter Values
params = {}
# Loop over system size
for outputs in range(1,3):
    for inputs in range(1,3):
        # Process Parameter 
        # System Gain
        params.update({"fmu.num["+str(outputs)+","+str(inputs)+",1]": k[outputs-1][inputs-1]})
        # System Lag
        for order in range(0,9):
            params.update({"fmu.den["+str(outputs)+","+str(inputs)+","+str(order+1)+"]": t[outputs-1][inputs-1][order]})
        #params.update({"fmu.den["+str(outputs)+","+str(inputs)+",9]": 1})
        # System Delay
        params.update({"fmu.delay["+str(outputs)+","+str(inputs)+"]": l[outputs-1][inputs-1]})

# Set Parameter and show for checking
sim.set(params)